In [1]:
import os
import shutil
import numpy as np
import pandas as pd

In [2]:
RAW_DATA = '../data/raw'
INTERIM_DATA = '../data/interim/'
PROCESSED_DATA ='../data/processed/'


TEST_DATA = '../data/interim/test/'
LIVE_DATA = '../data/interim/live/'
TRAIN_DATA = '../data/interim/train/'

In [3]:
TRAIN_DATA_PATH = os.path.join(TRAIN_DATA, 'dataset.csv')
TEST_DATA_PATH = os.path.join(TEST_DATA, 'dataset.csv')
LIVE_DATA_PATH = os.path.join(LIVE_DATA, 'dataset.csv')

In [4]:
RAW_DATA_PATH = os.path.join(RAW_DATA, 'dataset.csv')


DATASET = pd.read_csv(RAW_DATA_PATH, parse_dates=['Timestamp'])

DATASET.head(10)

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
0,2024-01-01 00:00:00,232.483571,5.124673,1.191402,0.399029,0.907766,46.614845,26.399566,0.0,1.055937,0,0,17.842767,64.523353,0.416156,1.233158
1,2024-01-01 00:15:00,229.308678,22.213519,5.093753,1.248030,0.831813,6.907170,18.555011,0.0,-3.910902,1,0,18.753698,49.671940,0.492334,5.247824
2,2024-01-01 00:30:00,233.238443,46.132217,10.759806,2.790464,0.999005,15.477640,12.905025,0.0,-0.197551,0,0,16.719325,48.472769,0.427785,10.580278
3,2024-01-01 00:45:00,237.615149,47.648972,11.322118,1.956965,0.896462,35.746398,28.937129,0.0,2.671921,0,0,15.364564,75.766776,0.487953,11.225946
4,2024-01-01 01:00:00,228.829233,7.410166,1.695663,0.624718,0.870681,27.555038,14.184648,0.0,3.826937,0,0,38.760333,61.384213,0.290078,1.696639
5,2024-01-01 01:15:00,228.829315,10.554009,2.415067,0.935853,0.970003,46.029339,25.887703,0.0,2.201227,0,0,25.969806,50.437474,0.469709,2.482321
6,2024-01-01 01:30:00,237.896064,5.219166,1.241619,0.368005,0.962855,21.536427,13.128721,0.0,-0.800185,1,0,31.406756,74.001033,0.255648,1.229065
7,2024-01-01 01:45:00,233.837174,44.090317,10.309955,1.500174,0.846779,11.041828,7.978982,0.0,4.361457,0,0,23.179631,44.945221,0.421238,10.776034
8,2024-01-01 02:00:00,227.652628,8.189880,1.864448,0.485411,0.968598,3.752725,3.927133,0.0,4.481335,0,0,24.576090,26.956974,0.431230,1.807544
9,2024-01-01 02:15:00,232.712800,9.855507,2.293503,0.437459,0.856682,24.417062,29.437511,0.0,0.468041,1,0,11.235522,26.669850,0.169219,2.297330


In [5]:
def standardTimeFormate(database, column):
    database[column] = pd.to_datetime(database[column])

In [6]:
COLUMN_NAME = 'Timestamp'


database = DATASET.copy()

standardTimeFormate(database, COLUMN_NAME)

database.set_index(COLUMN_NAME, inplace=True)

In [7]:
database = database[~database.index.duplicated(keep='first')]

database.sort_index(inplace=True)

database.sort_index(axis=1, inplace=True)

In [8]:
database.head(10)

,Current (A),Electricity Price (USD/kWh),Grid Supply (kW),Humidity (%),Overload Condition,Power Consumption (kW),Power Factor,Predicted Load (kW),Reactive Power (kVAR),Solar Power (kW),Temperature (°C),Transformer Fault,Voltage (V),Voltage Fluctuation (%),Wind Power (kW)
Timestamp,,,,,,,,,,,,,,,
2024-01-01 00:00:00,5.124673,0.416156,0.0,64.523353,0,1.191402,0.907766,1.233158,0.399029,46.614845,17.842767,0,232.483571,1.055937,26.399566
2024-01-01 00:15:00,22.213519,0.492334,0.0,49.671940,1,5.093753,0.831813,5.247824,1.248030,6.907170,18.753698,0,229.308678,-3.910902,18.555011
2024-01-01 00:30:00,46.132217,0.427785,0.0,48.472769,0,10.759806,0.999005,10.580278,2.790464,15.477640,16.719325,0,233.238443,-0.197551,12.905025
2024-01-01 00:45:00,47.648972,0.487953,0.0,75.766776,0,11.322118,0.896462,11.225946,1.956965,35.746398,15.364564,0,237.615149,2.671921,28.937129
2024-01-01 01:00:00,7.410166,0.290078,0.0,61.384213,0,1.695663,0.870681,1.696639,0.624718,27.555038,38.760333,0,228.829233,3.826937,14.184648
2024-01-01 01:15:00,10.554009,0.469709,0.0,50.437474,0,2.415067,0.970003,2.482321,0.935853,46.029339,25.969806,0,228.829315,2.201227,25.887703
2024-01-01 01:30:00,5.219166,0.255648,0.0,74.001033,1,1.241619,0.962855,1.229065,0.368005,21.536427,31.406756,0,237.896064,-0.800185,13.128721
2024-01-01 01:45:00,44.090317,0.421238,0.0,44.945221,0,10.309955,0.846779,10.776034,1.500174,11.041828,23.179631,0,233.837174,4.361457,7.978982
2024-01-01 02:00:00,8.189880,0.431230,0.0,26.956974,0,1.864448,0.968598,1.807544,0.485411,3.752725,24.576090,0,227.652628,4.481335,3.927133


In [9]:
database.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 50000 entries, 2024-01-01 00:00:00 to 2025-06-04 19:45:00
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Current (A)                  50000 non-null  float64
 1   Electricity Price (USD/kWh)  50000 non-null  float64
 2   Grid Supply (kW)             50000 non-null  float64
 3   Humidity (%)                 50000 non-null  float64
 4   Overload Condition           50000 non-null  int64  
 5   Power Consumption (kW)       50000 non-null  float64
 6   Power Factor                 50000 non-null  float64
 7   Predicted Load (kW)          50000 non-null  float64
 8   Reactive Power (kVAR)        50000 non-null  float64
 9   Solar Power (kW)             50000 non-null  float64
 10  Temperature (°C)             50000 non-null  float64
 11  Transformer Fault            50000 non-null  int64  
 12  Voltage (V)                  50000 non-null  float

In [10]:
def getTemporalFeatures(database):
    database['Hours'] = database.index.hour
    
    database['Day of Week'] = database.index.dayofweek
    
    database['Weekend'] = database.index.dayofweek.isin([5, 6]).astype(int)

In [11]:
getTemporalFeatures(database)

In [12]:
database.columns

Index(['Current (A)', 'Electricity Price (USD/kWh)', 'Grid Supply (kW)',
       'Humidity (%)', 'Overload Condition', 'Power Consumption (kW)',
       'Power Factor', 'Predicted Load (kW)', 'Reactive Power (kVAR)',
       'Solar Power (kW)', 'Temperature (°C)', 'Transformer Fault',
       'Voltage (V)', 'Voltage Fluctuation (%)', 'Wind Power (kW)', 'Hours',
       'Day of Week', 'Weekend'],
      dtype='str')

In [13]:
database.head(10)

,Current (A),Electricity Price (USD/kWh),Grid Supply (kW),Humidity (%),Overload Condition,Power Consumption (kW),Power Factor,Predicted Load (kW),Reactive Power (kVAR),Solar Power (kW),Temperature (°C),Transformer Fault,Voltage (V),Voltage Fluctuation (%),Wind Power (kW),Hours,Day of Week,Weekend
Timestamp,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,5.124673,0.416156,0.0,64.523353,0,1.191402,0.907766,1.233158,0.399029,46.614845,17.842767,0,232.483571,1.055937,26.399566,0,0,0
2024-01-01 00:15:00,22.213519,0.492334,0.0,49.671940,1,5.093753,0.831813,5.247824,1.248030,6.907170,18.753698,0,229.308678,-3.910902,18.555011,0,0,0
2024-01-01 00:30:00,46.132217,0.427785,0.0,48.472769,0,10.759806,0.999005,10.580278,2.790464,15.477640,16.719325,0,233.238443,-0.197551,12.905025,0,0,0
2024-01-01 00:45:00,47.648972,0.487953,0.0,75.766776,0,11.322118,0.896462,11.225946,1.956965,35.746398,15.364564,0,237.615149,2.671921,28.937129,0,0,0
2024-01-01 01:00:00,7.410166,0.290078,0.0,61.384213,0,1.695663,0.870681,1.696639,0.624718,27.555038,38.760333,0,228.829233,3.826937,14.184648,1,0,0
2024-01-01 01:15:00,10.554009,0.469709,0.0,50.437474,0,2.415067,0.970003,2.482321,0.935853,46.029339,25.969806,0,228.829315,2.201227,25.887703,1,0,0
2024-01-01 01:30:00,5.219166,0.255648,0.0,74.001033,1,1.241619,0.962855,1.229065,0.368005,21.536427,31.406756,0,237.896064,-0.800185,13.128721,1,0,0
2024-01-01 01:45:00,44.090317,0.421238,0.0,44.945221,0,10.309955,0.846779,10.776034,1.500174,11.041828,23.179631,0,233.837174,4.361457,7.978982,1,0,0
2024-01-01 02:00:00,8.189880,0.431230,0.0,26.956974,0,1.864448,0.968598,1.807544,0.485411,3.752725,24.576090,0,227.652628,4.481335,3.927133,2,0,0


In [14]:
ROWS = len(DATASET)
TRAIN_RATIO = int(ROWS * 0.70)
TEST_RATIO = int(ROWS * 0.85)

In [15]:
os.makedirs(os.path.join(INTERIM_DATA, "train"), exist_ok=True)
os.makedirs(os.path.join(INTERIM_DATA, "test"), exist_ok=True)
os.makedirs(os.path.join(INTERIM_DATA, "live"), exist_ok=True)

In [16]:
TRAINING_DATA = database.iloc[:TRAIN_RATIO]
TESTING_DATA = database.iloc[TRAIN_RATIO:TEST_RATIO]
STREAMING_DATA = database.iloc[TEST_RATIO:]


TRAINING_DATA.to_csv(TRAIN_DATA_PATH)
TESTING_DATA.to_csv(TEST_DATA_PATH)
STREAMING_DATA.to_csv(LIVE_DATA_PATH)

In [17]:
os.makedirs(os.path.join(PROCESSED_DATA, "train"), exist_ok=True)
os.makedirs(os.path.join(PROCESSED_DATA, "test"), exist_ok=True)

In [18]:
shutil.copy(TRAIN_DATA_PATH, os.path.join(PROCESSED_DATA, 'train/dataset.csv'))

'../data/processed/train/dataset.csv'

In [19]:
shutil.copy(TEST_DATA_PATH, os.path.join(PROCESSED_DATA, 'test/dataset.csv'))

'../data/processed/test/dataset.csv'